## Tp2 ift 870
Quentin Levieux<br>
Frederic Giasson

In [1]:
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import numbers

In [2]:
package = pd.read_csv("Package2.csv",sep=";", error_bad_lines=False, encoding ='latin1')
display("Dimension de package.csv : ",package.shape)
product = pd.read_csv("Product2.csv",sep=";", error_bad_lines=False, encoding ='latin1')
print("Dimension de product.csv: ",product.shape)
print("Nombre d'attributs de package:", len(package.columns))
print("Nombre d'attributs de product:", len(product.columns))
print("\nColonne de package: \n",package.columns.values)
print("\nColonne de product: \n",product.columns.values)
commonAttributs = list(set(package).intersection(product))
print("\nAttributs communs au deux tables :\n",commonAttributs)

'Dimension de package.csv : '

(173887, 8)

Dimension de product.csv:  (93238, 20)
Nombre d'attributs de package: 8
Nombre d'attributs de product: 20

Colonne de package: 
 ['PRODUCTID' 'PRODUCTNDC' 'NDCPACKAGECODE' 'PACKAGEDESCRIPTION'
 'STARTMARKETINGDATE' 'ENDMARKETINGDATE' 'NDC_EXCLUDE_FLAG'
 'SAMPLE_PACKAGE']

Colonne de product: 
 ['PRODUCTID' 'PRODUCTNDC' 'PRODUCTTYPENAME' 'PROPRIETARYNAME'
 'PROPRIETARYNAMESUFFIX' 'NONPROPRIETARYNAME' 'DOSAGEFORMNAME' 'ROUTENAME'
 'STARTMARKETINGDATE' 'ENDMARKETINGDATE' 'MARKETINGCATEGORYNAME'
 'APPLICATIONNUMBER' 'LABELERNAME' 'SUBSTANCENAME'
 'ACTIVE_NUMERATOR_STRENGTH' 'ACTIVE_INGRED_UNIT' 'PHARM_CLASSES'
 'DEASCHEDULE' 'NDC_EXCLUDE_FLAG' 'LISTING_RECORD_CERTIFIED_THROUGH']

Attributs communs au deux tables :
 ['STARTMARKETINGDATE', 'PRODUCTID', 'ENDMARKETINGDATE', 'PRODUCTNDC', 'NDC_EXCLUDE_FLAG']


In [3]:
product.head(10)

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,NaN,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,NDA,NDA018781,10,WATER,1,mL/mL,NaN,NaN,N,20201231.0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20211231.0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0
5,NaN,0002-1445,HUMAN PRESCRIPTION DRUG,TALTZ,NaN,ixekizumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20160322,NaN,BLA,BLA125521,10,IXEKIZUMAB,80,mg/mL,"Interleukin-17A Antagonist [EPC],Interleukin-1...",NaN,N,20201231.0
6,NaN,0002-2377,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0
7,NaN,0002-3115,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20190604,NaN,BLA,BLA761063,10,GALCANEZUMAB,100,mg/mL,NaN,NaN,N,20201231.0
8,NaN,0002-3227,HUMAN PRESCRIPTION DRUG,Strattera,NaN,Atomoxetine hydrochloride,CAPSULE,ORAL,20021126,NaN,NDA,NDA021411,10,ATOMOXETINE HYDROCHLORIDE,10,mg/1,"Norepinephrine Reuptake Inhibitor [EPC],Norepi...",NaN,N,20211231.0
9,NaN,0002-3228,HUMAN PRESCRIPTION DRUG,Strattera,NaN,Atomoxetine hydrochloride,CAPSULE,ORAL,20021126,NaN,NDA,NDA021411,10,ATOMOXETINE HYDROCHLORIDE,25,mg/1,"Norepinephrine Reuptake Inhibitor [EPC],Norepi...",NaN,N,20211231.0


In [4]:
package.head(8)

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710,NaN,N,N
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601,NaN,N,N
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601,NaN,N,N
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107,NaN,N,Y
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-80,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,20141107,NaN,N,N
5,0002-1434_42a80046-fd68-4b80-819c-a443b7816edb,0002-1434,0002-1434-61,2 SYRINGE in 1 CARTON (0002-1434-61) > .5 mL ...,20141107,NaN,N,Y
6,0002-1434_42a80046-fd68-4b80-819c-a443b7816edb,0002-1434,0002-1434-80,4 SYRINGE in 1 CARTON (0002-1434-80) > .5 mL ...,20141107,NaN,N,N
7,0002-1436_dd7f66eb-66e1-4ad0-a593-417dbf2c74e2,0002-1436,0002-1436-11,1 SYRINGE in 1 CARTON (0002-1436-11) > 1 mL i...,20180927,NaN,N,N


# Question 1

## Types des attributs

In [5]:
print("Types des attributs pour package :\n\n",package.dtypes)
print("\nTypes des attributs pour product :\n\n",product.dtypes)

Types des attributs pour package :

 PRODUCTID              object
PRODUCTNDC             object
NDCPACKAGECODE         object
PACKAGEDESCRIPTION     object
STARTMARKETINGDATE      int64
ENDMARKETINGDATE      float64
NDC_EXCLUDE_FLAG       object
SAMPLE_PACKAGE         object
dtype: object

Types des attributs pour product :

 PRODUCTID                            object
PRODUCTNDC                           object
PRODUCTTYPENAME                      object
PROPRIETARYNAME                      object
PROPRIETARYNAMESUFFIX                object
NONPROPRIETARYNAME                   object
DOSAGEFORMNAME                       object
ROUTENAME                            object
STARTMARKETINGDATE                    int64
ENDMARKETINGDATE                    float64
MARKETINGCATEGORYNAME                object
APPLICATIONNUMBER                    object
LABELERNAME                          object
SUBSTANCENAME                        object
ACTIVE_NUMERATOR_STRENGTH            object
ACTIVE_INGR

### Valeurs Manquantes

In [6]:
manquantePackage = package[package.isnull().any(axis=1)]
#display(manquantePackage)
print(manquantePackage.shape)

manquanteProduct = product[product.isnull().any(axis=1)]
#display(manquanteProduct)
print(manquanteProduct.shape)

print("Valeurs manquante par colonne dans package:\n")
for col in package.columns:
    nb = package[[col]].isna().sum()
    print(nb)
print("Nombre total de rangees dans package :", len(package))

print("\nValeurs manquante par colonne dans product:\n")
for col in product.columns:
    nb = product[[col]].isna().sum()
    print(nb)
print("Nombre total de rangees dans product:", len(product))

(167507, 8)
(93238, 20)
Valeurs manquante par colonne dans package:

PRODUCTID    0
dtype: int64
PRODUCTNDC    1500
dtype: int64
NDCPACKAGECODE    2346
dtype: int64
PACKAGEDESCRIPTION    0
dtype: int64
STARTMARKETINGDATE    0
dtype: int64
ENDMARKETINGDATE    167431
dtype: int64
NDC_EXCLUDE_FLAG    0
dtype: int64
SAMPLE_PACKAGE    0
dtype: int64
Nombre total de rangees dans package : 173887

Valeurs manquante par colonne dans product:

PRODUCTID    1560
dtype: int64
PRODUCTNDC    0
dtype: int64
PRODUCTTYPENAME    0
dtype: int64
PROPRIETARYNAME    6
dtype: int64
PROPRIETARYNAMESUFFIX    83075
dtype: int64
NONPROPRIETARYNAME    4
dtype: int64
DOSAGEFORMNAME    0
dtype: int64
ROUTENAME    1932
dtype: int64
STARTMARKETINGDATE    0
dtype: int64
ENDMARKETINGDATE    88915
dtype: int64
MARKETINGCATEGORYNAME    0
dtype: int64
APPLICATIONNUMBER    13097
dtype: int64
LABELERNAME    0
dtype: int64
SUBSTANCENAME    2309
dtype: int64
ACTIVE_NUMERATOR_STRENGTH    2309
dtype: int64
ACTIVE_INGRED_UNIT  

### Incohérences dans package

In [7]:
#Verifier que productId possède au moins 1 tiret 
productIdNotNull = package[~package['PRODUCTID'].isnull()]
arrayIndexInvalidProductIdForPackage = []

for index, row in productIdNotNull.iterrows():
    if "-" not in row['PRODUCTID']:
        arrayIndexInvalidProductIdForPackage.append(index)
print("Nombres de PRODUCTID n'ayant pas de tiret:",len(arrayIndexInvalidProductIdForPackage))

Nombres de PRODUCTID n'ayant pas de tiret: 0


In [8]:
ndcPackageCodeNotNull = package[~package['NDCPACKAGECODE'].isnull()]
arrayIndexInvalidNDCPackageCode = []

for index, row in ndcPackageCodeNotNull.iterrows():
    #Si le productndc ne contient pas de tiret
    if not row['NDCPACKAGECODE'].count("-")== 2:
        arrayIndexInvalidNDCPackageCode.append(index)

    elif not row['NDCPACKAGECODE'].replace('-','').isnumeric():
        arrayIndexInvalidNDCPackageCode.append(index)
        
print("Nombres de NDCPACKAGECODE n'ayant pas 2 tirets ou contenant des lettres:",len(arrayIndexInvalidNDCPackageCode))

Nombres de NDCPACKAGECODE n'ayant pas 2 tirets ou contenant des lettres: 154


In [9]:
#Verifier que le productNDC est composé de nombre et contient un tiret comme indiqué dans la documentation
productNDCNotNull = package[~package['PRODUCTNDC'].isnull()]
arrayIndexInvalidProductNDCForPackage = []

for index, row in productNDCNotNull.iterrows():
    #Si le productndc ne contient pas de tiret
    if not row['PRODUCTNDC'].count("-")== 1:
        arrayIndexInvalidProductNDCForPackage.append(index)

    elif not row['PRODUCTNDC'].replace('-','').isnumeric():
        arrayIndexInvalidProductNDCForPackage.append(index)
    
print("Nombres de PRODUCTNDC n'ayant pas de tiret ou contenant des lettres:",len(arrayIndexInvalidProductNDCForPackage))

Nombres de PRODUCTNDC n'ayant pas de tiret ou contenant des lettres: 519


In [10]:
#Verifier que PRODUCTNDC apparait au debut de PRODUCTID
#Dans le cas ou le PRODUCTNDC et le PRODUCTID existent
productNDCAndProductIdNotNull = package[(~package['PRODUCTNDC'].isnull()) & (~package['PRODUCTID'].isnull())]

arrayIndexIncoherentProductNDCAndProductIdForPackage = []
compteurIncoherence = 0
for index, row in productNDCAndProductIdNotNull.iterrows() :
    productndc = row['PRODUCTNDC']
    productid = row['PRODUCTID']
    if productndc != productid[:len(productndc)]:
        arrayIndexIncoherentProductNDCAndProductIdForPackage.append(index)
        compteurIncoherence += 1
        
print("Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC : ", compteurIncoherence)
print(len(arrayIndexIncoherentProductNDCAndProductIdForPackage))

Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC :  519
519


In [11]:
#Dans le cas ou le PRODUCTNDC et le NDCPackageCode existent
productNDCAndNDCPackageCodeNotNull = package[(~package['PRODUCTNDC'].isnull()) & (~package['NDCPACKAGECODE'].isnull())]

arrayIndexIncoherentProductNDCAndNDCPackageCode = []
compteurIncoherence = 0
for index, row in productNDCAndNDCPackageCodeNotNull.iterrows() :
    productndc = row['PRODUCTNDC']
    ndcpkgcode = row['NDCPACKAGECODE']
    if productndc != ndcpkgcode[:len(productndc)]:
        arrayIndexIncoherentProductNDCAndNDCPackageCode.append(index)
        compteurIncoherence += 1
        
print("Nombre de PRODUCTNDC contenu dans NDCPACKAGECODE différent du PRODUCTNDC : ", compteurIncoherence)

Nombre de PRODUCTNDC contenu dans NDCPACKAGECODE différent du PRODUCTNDC :  674


In [12]:
#Aucune valeur manquante dans package pour la colonne STARTMARKETINGDATE donc on peut utiliser directement package
dateFormatNonValide = 0
arrayIndexInvalidStartMktgDateForPackage = []
for index, row in package.iterrows():
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(row['STARTMARKETINGDATE'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        arrayIndexInvalidStartMktgDateForPackage.append(index)
        dateFormatNonValide +=1
        
print("Nombre de dates de STARTMARKETINGDATE invalides : ",dateFormatNonValide)

Nombre de dates de STARTMARKETINGDATE invalides :  243


In [13]:
endMarketingDateNotnull = package[(~package['ENDMARKETINGDATE'].isnull())]

dateFormatNonValide = 0
for index in range(0,len(endMarketingDateNotnull)):
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(endMarketingDateNotnull.iloc[index]['ENDMARKETINGDATE'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        dateFormatNonValide +=1
        
print("Nombre de dates de ENDMARKETINGDATE invalides : ",dateFormatNonValide)

Nombre de dates de ENDMARKETINGDATE invalides :  0


In [14]:
#Verifier que la date start soit avant ou égale à la date end (cependant date start peut être non valide)
datesIncoherentes = 0
arrayIndexDatesIncoherentesForPackage = []

for index, row in endMarketingDateNotnull.iterrows():
    dateEnd = pd.to_datetime(row['ENDMARKETINGDATE'])
    dateStart = pd.to_datetime(row['STARTMARKETINGDATE'])
    
    if dateEnd < dateStart :
        arrayIndexDatesIncoherentesForPackage.append(index)
        datesIncoherentes +=1
print("Nombre de dates incohérentes tel que start soit apres end, ou soit la date start est invalide : ",datesIncoherentes)

Nombre de dates incohérentes tel que start soit apres end, ou soit la date start est invalide :  18


In [15]:
#incoherence dans la table package
#SAMPLE_PACKAGE peut prendre une valeur parmis Y ou N
samplePackage = package[(package['SAMPLE_PACKAGE'] != 'Y') & (package['SAMPLE_PACKAGE'] != 'N')]

print("Nombre de ligne avec une valeur autre que Y ou N dans la colonne sample_Package : ",len(samplePackage))

#NDC_EXCLUDE_FLAG peut prendre une valeur parmis Y, N, E ou I
ndc_exclude_flag = package[(package['NDC_EXCLUDE_FLAG'] != 'Y') & (package['NDC_EXCLUDE_FLAG'] != 'N') & (package['NDC_EXCLUDE_FLAG'] != 'E') & (package['NDC_EXCLUDE_FLAG'] != 'I')]
print("Nombre de rangee avec une valeur autre que Y, N, E ou I dans la colonne ndc_exclude_flag : ",len(ndc_exclude_flag))

Nombre de ligne avec une valeur autre que Y ou N dans la colonne sample_Package :  0
Nombre de rangee avec une valeur autre que Y, N, E ou I dans la colonne ndc_exclude_flag :  0


En ce qui attrait au colonnes <b>PRODUCTID PRODUCTNDC NDCPACKAGECODE PACKAGEDESCRIPTION</b> de la table PACKAGE, il est difficile de d'avoir un critère qui permet de savoir si il y a une incoherence dans les données de la colonne. Pour ce qui est des autres colonnes, force est de constater qu'il n'y a pas d'incoherence flagrante dans les données intra colonnes. En effet, d'autant plus que les données de ces colonnes sont du type OBJECT, ils peuvent donc prendre n'importe quelle forme.

### Incohérences dans product

In [16]:
#Verifier que productId possède au moins 1 tiret 
productIdNotNull = product[~product['PRODUCTID'].isnull()]
arrayIndexInvalidProductIdForProduct = []

for index, row in productIdNotNull.iterrows():
    if "-" not in row['PRODUCTID']:
        arrayIndexInvalidProductIdForProduct.append(index)
print("Nombres de PRODUCTID n'ayant pas de tiret:",len(arrayIndexInvalidProductIdForProduct))

Nombres de PRODUCTID n'ayant pas de tiret: 0


In [17]:
deaScheduleNotNull = product[~product['DEASCHEDULE'].isnull()] 

arrayIndexInvalidDeaSchedule = []

for index, row in deaScheduleNotNull.iterrows():
    if row['DEASCHEDULE'] not in ['CI', 'CII', 'CIII', 'CIV','CV']:
        arrayIndexInvalidDeaSchedule.append(index)
print("Nombre de DEASCHEDULE n'ayant pas de valeur valide :",len(arrayIndexInvalidDeaSchedule))


Nombre de DEASCHEDULE n'ayant pas de valeur valide : 0


In [39]:
#Verifier que le productNDC est composé de nombre et contient un tiret comme indiqué dans la documentation
productNDCNotNull = product[~product['PRODUCTNDC'].isnull()]
arrayIndexInvalidProductNDCForProduct = []

for index, row in productNDCNotNull.iterrows():
    #Si le productndc ne contient pas de tiret
    if "-" not in row['PRODUCTNDC']:
        arrayIndexInvalidProductNDCForProduct.append(index)
        
    elif not row['PRODUCTNDC'].replace('-','').isnumeric():
        arrayIndexInvalidProductNDCForProduct.append(index)
    
print("Nombres de PRODUCTNDC n'ayant pas de tiret ou contenant des lettres:",len(arrayIndexInvalidProductNDCForProduct))

Nombres de PRODUCTNDC n'ayant pas de tiret ou contenant des lettres: 0


In [40]:
#Dans le cas ou le PRODUCTNDC et le PRODUCTID existent
productNDCAndProductIdNotNull = product[(~product['PRODUCTNDC'].isnull()) & (~product['PRODUCTID'].isnull())]

arrayIndexIncoherentProductNDCAndProductIdForProduct = []
compteurIncoherence = 0
for index, row in productNDCAndProductIdNotNull.iterrows() :
    productndc = row['PRODUCTNDC']
    productid = row['PRODUCTID']
    if productndc != productid[:len(productndc)]:
        arrayIndexIncoherentProductNDCAndProductIdForProduct.append(index)
        compteurIncoherence += 1
        
print("Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC : ", compteurIncoherence) 

Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC :  0


In [41]:
#Dans le cas ou le MARKETINGCATEGORYNAME et le APPLICATIONNUMBER existent
#On ne teste pas sur MARKETINGCATEGORYNAME car il y a aucune valeur manquante
applicationNumberNotNull = product[(~product['APPLICATIONNUMBER'].isnull())]

compteurIncoherence = 0
arrayIndexIncoherentMarketingCategoryNameAndApplicationNumber = []
for index, row in applicationNumberNotNull.iterrows() :
    marketingctgname = row['MARKETINGCATEGORYNAME']
    applicationnumber = row['APPLICATIONNUMBER']
    
    if marketingctgname != applicationnumber[:len(marketingctgname)]:
        #Condition donner sur le site que si  Marketing Category is OTC Monograph Final or OTC Monograph Not Final, alor application nb sera partXXXX
        if str(marketingctgname[:3]) == "OTC" and str(applicationnumber[:4])=="part" or str(marketingctgname[:3]) == str(applicationnumber[:3]):
            #print(marketingctgname[0:3], applicationnumber[:4])
            continue
        else:    
            arrayIndexIncoherentMarketingCategoryNameAndApplicationNumber.append(index)
            compteurIncoherence += 1
print("Nombre de MARKETINGCATEGORYNAME contenu dans APPLICATIONNUMBER différent du MARKETINGCATEGORYNAME : ", compteurIncoherence)   

Nombre de MARKETINGCATEGORYNAME contenu dans APPLICATIONNUMBER différent du MARKETINGCATEGORYNAME :  124


In [21]:
#NDC_EXCLUDE_FLAG peut prendre une valeur parmis Y, N, E ou I
ndc_exclude_flag = product[(product['NDC_EXCLUDE_FLAG'] != 'Y') & (product['NDC_EXCLUDE_FLAG'] != 'N') & (product['NDC_EXCLUDE_FLAG'] != 'E') & (product['NDC_EXCLUDE_FLAG'] != 'I')]
print("Nombre de ligne avec une valeur autre que Y, N, E ou I dans la colonne ndc_exclude_flag : ",len(ndc_exclude_flag))

Nombre de ligne avec une valeur autre que Y, N, E ou I dans la colonne ndc_exclude_flag :  0


In [22]:
#MARKETINGCATEGORYNAME peut prendre une valeur parmis NDA, ANDA, BLA, OTC Monograph, ou Unapproved Drug
marketingctgname = product[(product['MARKETINGCATEGORYNAME'] != 'NDA') & (product['MARKETINGCATEGORYNAME'] != 'ANDA') & (product['MARKETINGCATEGORYNAME'] != 'BLA')]
print("Nombre de ligne avec une valeur autre que NDA, ANDA, BLA dans la colonne MARKETINGCATEGORYNAME : ",len(marketingctgname))

Nombre de ligne avec une valeur autre que NDA, ANDA, BLA dans la colonne MARKETINGCATEGORYNAME :  44782


In [23]:
#Aucune valeur manquante dans package pour la colonne STARTMARKETINGDATE donc on peut utiliser directement product
dateFormatNonValide = 0
for index in range(0,len(product)):
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(product.iloc[index]['STARTMARKETINGDATE'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        dateFormatNonValide +=1
        
print("Nombre de dates de STARTMARKETINGDATE invalides : ",dateFormatNonValide)

Nombre de dates de STARTMARKETINGDATE invalides :  0


In [42]:
endMarketingDateNotnull = product[(~product['ENDMARKETINGDATE'].isnull())]

arrayIndexInvalidEndMarketingDateForProduct = []
dateFormatNonValide = 0
for index, row in endMarketingDateNotnull.iterrows():
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(row['ENDMARKETINGDATE'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        arrayIndexInvalidEndMarketingDateForProduct.append(index)
        dateFormatNonValide +=1
        
print("Nombre de dates de ENDMARKETINGDATE invalide : ",dateFormatNonValide)

Nombre de dates de ENDMARKETINGDATE invalide :  0


In [25]:
#Verifier que la date start soit avant ou égale à la date end (cependant end start peut être non valide car 1 trouvée)
datesIncoherentes = 0
for index in range(0,len(endMarketingDateNotnull)):
    dateEnd = pd.to_datetime(endMarketingDateNotnull.iloc[index]['ENDMARKETINGDATE'])
    dateStart = pd.to_datetime(endMarketingDateNotnull.iloc[index]['STARTMARKETINGDATE'])
    
    if dateEnd < dateStart :
        datesIncoherentes +=1
print("Nombre de dates incohérentes tel que start soit apres end, ou soit la date start est invalide : ",datesIncoherentes)

Nombre de dates incohérentes tel que start soit apres end, ou soit la date start est invalide :  0


In [26]:
listrecordcertifNotNull = product[~product['LISTING_RECORD_CERTIFIED_THROUGH'].isnull()]

dateFormatNonValide = 0
for index in range(0,len(listrecordcertifNotNull)):
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(listrecordcertifNotNull.iloc[index]['LISTING_RECORD_CERTIFIED_THROUGH'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        dateFormatNonValide +=1
print("Nombre de dates de LISTING_RECORD_CERTIFIED_THROUGH invalides: ", dateFormatNonValide)

Nombre de dates de LISTING_RECORD_CERTIFIED_THROUGH invalides:  0


In [27]:
hh = product[~product['DEASCHEDULE'].isnull()]
print("Differentes possibilites pour DEASCHEDULE :",hh['DEASCHEDULE'].unique())

jj = product[~product['ACTIVE_INGRED_UNIT'].isnull()]
print("Differentes possibilites pour ACTIVE_INGRED_UNIT :",jj['ACTIVE_INGRED_UNIT'].unique())

kk = product[~product['ACTIVE_NUMERATOR_STRENGTH'].isnull()]
erreurActiveNumerator= [isinstance(x, numbers.Number) for x in kk]
print("Differentes possibilites pour ACTIVE_NUMERATOR_STRENGTH :",kk['ACTIVE_NUMERATOR_STRENGTH'].unique())
print("Il y a ",len(erreurActiveNumerator), "identifiants de ACTIVE_NUMERATOR_STRENGTH qui ne sont pas des nombres")

ll = product[~product['ROUTENAME'].isnull()]
print("Nombres de differentes possibilites pour ROUTENAME :",len(ll['ROUTENAME'].unique()))

zz = product[~product['DOSAGEFORMNAME'].isnull()]
print("Nombres de differentes possibilites pour DOSAGEFORMNAME :",len(zz['DOSAGEFORMNAME'].unique()))

cc = product[~product['PROPRIETARYNAMESUFFIX'].isnull()]
nameSuffix = []
for date in cc['PROPRIETARYNAMESUFFIX'].unique():
    if any(char.isdigit() for char in date):
           nameSuffix.append(date)
print("Nombres de differentes possibilites pour PROPRIETARYNAMESUFFIX :",len(nameSuffix))

Differentes possibilites pour DEASCHEDULE : ['CV' 'CIV' 'CIII' 'CII']
Differentes possibilites pour ACTIVE_INGRED_UNIT : ['mL/mL' 'mCi/mL' 'mg/.5mL' ... 'g/30mL; g/30mL; g/30mL; 1/30mL' '1/25mL'
 'mg/6g']
Differentes possibilites pour ACTIVE_NUMERATOR_STRENGTH : ['1' '51' '.75' ... '33.2' '415' '20; 25; 33']
Il y a  20 identifiants de ACTIVE_NUMERATOR_STRENGTH qui ne sont pas des nombres
Nombres de differentes possibilites pour ROUTENAME : 180
Nombres de differentes possibilites pour DOSAGEFORMNAME : 134
Nombres de differentes possibilites pour PROPRIETARYNAMESUFFIX : 1455


Pour la colonne <b>DEASCHEDULE</b>, nous ne connaissons pas les valeurs des attributs il est donc difficile de savoir si il y a une incoherence.
<br><br>Pour la colonne <b>PHARM_CLASSES</b>, nous ne pouvons pas evaluer les incoherences en raison des valeurs de la colonne.
<br><br>Pour la colonne <b>ACTIVE_INGRED_UNIT</b>, il y a plusieurs valeurs, cependant en les analysant visuellement, il ne semble pas y avoir d'incoherences.
<br><br>Pour la colonne <b>ACTIVE_NUMERATOR_STRENGTH</b>, il y a des valeurs qui ne sont pas des nombres. Nous ne savons pas si ce sont des incoherences cependant, nous estimons pertinent de le souligner au cas ou cela pourrait poserait probleme plus tard.
<br><br>Pour la colonne <b>SUBSTANCENAME</b> nous ne pouvons pas savoir si les attributs qui sont dans les colonnes sont bons puisque nous n'avons pas de banque permettant de valider. Il en est de meme pour <b>MARKETINGCATEGORYNAME APPLICATIONNUMBER LABELERNAME NONPROPRIETARYNAME PRODUCTID PRODUCTNDC PRODUCTTYPENAME PROPRIETARYNAME</b>, ce sont tous de objets, nous n'avons donc pas beaucoup de details quant a la composition des attributs.
<br><br>Pour la colonne <b>PROPRIETARYNAMESUFFIX</b> nous voyons qu'il y a 1455 elements que leur attribut contient un chiffre dans le suffix. Nous ne pouvons donc pas affirmer que de contenir un chiffre est un incoherence malgre que ce sont des suffix.

#### Conclusion 
Nous avons donc analysé toutes les colonnes afin de voir si il n'y avait pas d'erreurs ou d'incohérence dans les éléments de ces colonnes. Pour les dates nous avons vérifié qu'elles comportaient un mois entre 1 et 12 et un jour entre 1 et 31. Nous ne tenons pas compte de la variance du nombre de jours entre les mois. Lorsque la colonne comportait des objets qui par le nom nous permettait de croire que ce devrait être soi seulement composé de chiffres ou seulement composé de lettres, nous avons vérifié si c'était le cas et compté les nombre d'exceptions afin de vérifier si elles étaient répandues ou pas. Force est de constaté suite à notre analyse qu'il ne semble pas y avoir d'incohérence intra-attributs.

## NOTES 

Donner un sens à nos variables, plus simple pour la relecture

Les incohérences qu'il faudrait qu'on verifie : 

**Package :**

- dfdsfd
- qsdfdqsfdq
- [x] ProductNDC present dans ProductID
- [x] NDCPackageCode il y a ProductNDC dedans au début 
- [x] SamplePackage E {Y, N}
- [x] NDCExcludeFlag E {Y, N, E, I}
- [x] start et end marketindate ils doivent être cohérent l'un doit être avant l'autre
- [x] verifier format de date valide pour end et start

**Product :**

- [x] ProductNDC present dans ProductID
- [x] MarkekingCategoryName est au début d'ApplicationNumber A REVOIR NORMALEMENT ON NE PEUT RIEN MONTRER
- [ ] MarkekingCategoryName E {NDA, ANDA, BLA,  OTC Monograph, Unapproved Drug} A REVOIR NORMALEMENT ON NE PEUT RIEN MONTRER
- [x] NDCExcludeFlag E {Y, N, E, I}
- [x] start et end marketindate ils doivent être cohérent l'un doit être avant l'autre
- [x] verifier format de date valide pour en, start et listingRecord

#### C'est pour la Q6

**Cohérence entre les deux :**

- [ ] Dans les deux tables verifier que le couple ProductID, ProductNDC doit être le même
- [ ] start du package peut pas sortir avant le produit donc doit être avant le start du product  ??? A confirmer mais je pense
- [ ] ndcexcludeflag il faut qu’il soit cohérent entre les deux table, si le produit est exclu, le package aussi 

## Question2
Comme nous l'avons vu à la question 1, il y a certaines attributs qui sont reliés, qui se complète. Ci-dessous nous allons énumérer ces relations.

### Table Package
Les éléments suivants ne sont pas directement énuméré dans la table mais sont utilisés par 1 ou plusieurs éléments : SPL documentID, labeler code(labelerName dans la table Product), product code et package code de la National Drug Code number.
   - ProductID est le produit de la concaténation entre NDCproduct et SPL DocumentID.
   - Le productNDC est la concaténation entre le labelerCode et le productCode, ils sont séparés par des tirets. 
   - Le NDCPackageCode est le produit du labeler code, product code et package, ils sont séparés par des tirets. Deux de ses trois éléments sont partagés avec le productNDC.
   - Les autres éléments ne semblent pas avoir de liens avec d'autres attributs de cette table.

### Table product

Les éléments suivants ne sont pas directement énuméré dans la table mais sont utilisés par 1 ou plusieurs éléments : SPL documentID, product code et package code de la National Drug Code number.

   - **ProductID** est le produit de la concaténation entre NDCproduct et SPL DocumentID.
   - Le **productNDC** est la concaténation entre le labelerCode et le productCode, ils sont séparés par des tirets.
   - **ProprietaryNameSuffix** est composé de **ProprietaryName** suivit d'un suffix servant à distinguer un produit par caractéristiques.
   - L'attribut **MarketingCategoryName** aura l'une des étiquettes suivantes : <i>NDA/ANDA/BLA, OTC Monograph ou Unapproved Drug</i>. **ApplicationNumber** est le produit le MarketingCategoryName en ce sens qu'un code lui sera attribué selon la catégorie Marketing assignée.
   - L'attribut **SubstanceName** énumère les ingrédients actifs du produit. **StrengthNumber, StrengthUnit** est respectivement la force des ingrédients actifs identifiés et l'unité de mesure, l'ordre d'apparition des ingrédients est le même dans les deux catégories. L'attribut**Pharm_classes** est l'identifiant de la categorie pharmaceutique des substance identifié dans SubstanceName, l'ordre d'apparition est le même que dans les autres attributs. **DIRE QU ON EN FERA RIEN**

# Question 3

In [31]:
"""#Verifier que PRODUCTNDC apparait au debut de PRODUCTID
#Dans le cas ou le PRODUCTNDC et le PRODUCTID existent
productNDCAndProductIdNotNul = package[(~package['PRODUCTNDC'].isnull()) & (~package['PRODUCTID'].isnull())]

compteurIncoherence = 0
for index, row in productNDCAndProductIdNotNul.iterrows() :
    productndc = row['PRODUCTNDC']
    productid = row['PRODUCTID']
    if productndc != productid[:len(productndc)]:
        compteurIncoherence += 1
        
print("Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC : ", compteurIncoherence)  """  

'#Verifier que PRODUCTNDC apparait au debut de PRODUCTID\n#Dans le cas ou le PRODUCTNDC et le PRODUCTID existent\nproductNDCAndProductIdNotNul = package[(~package[\'PRODUCTNDC\'].isnull()) & (~package[\'PRODUCTID\'].isnull())]\n\ncompteurIncoherence = 0\nfor index, row in productNDCAndProductIdNotNul.iterrows() :\n    productndc = row[\'PRODUCTNDC\']\n    productid = row[\'PRODUCTID\']\n    if productndc != productid[:len(productndc)]:\n        compteurIncoherence += 1\n        \nprint("Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC : ", compteurIncoherence)  '

In [32]:
#Correction de ProductNDC et ProductId
#Il y a des cas ou il faut transformer la date et le mois en chiffre dans le bon ordre ex) 05-juin->0006-0005, nous en avons corrige 10
correction =""
for ind in arrayIndexIncoherentProductNDCAndProductIdForPackage:
    correction=""
    if str(package.iloc[ind]["PRODUCTNDC"][3:8]).isalpha():
        mois = str(package.iloc[ind]["PRODUCTNDC"][3:8])
        jour = str(package.iloc[ind]["PRODUCTNDC"][0:2])
        if mois == "sept":
            correction+="0009-00"+jour
        elif mois == "juin":
            correction+="0006-00"+jour
        package.loc[ind, "PRODUCTNDC"] = correction
        arrayIndexIncoherentProductNDCAndProductIdForPackage.remove(ind)
        
#Les incoherences qu'il reste a corriger(Je n'y vois aucune logique)
display(package.iloc[6913][["PRODUCTID","PRODUCTNDC"]])
#for ind in arrayIndexIncoherentProductNDCAndProductIdForPackage:
    # display(package.iloc[ind][["PRODUCTID","PRODUCTNDC"]])

PRODUCTID     0220-2997_8590fcc9-c0c3-f43b-e053-2991aa0a692e
PRODUCTNDC                                         0220-2997
Name: 6913, dtype: object

In [33]:
#Toute les erreurs sont la meme, l'annee est ecrit 2997 au lieu de 1997, nous corrigeons 243 erreurs
for date in arrayIndexInvalidStartMktgDateForPackage:
    nDate = "1"+str(package.iloc[date]["STARTMARKETINGDATE"])[1:]
    package.loc[date,"STARTMARKETINGDATE"]=nDate
    arrayIndexInvalidStartMktgDateForPackage.remove(date)

In [34]:
#Correction de l'erreur endMarketingDate
dd = product.iloc[arrayIndexInvalidEndMarketingDateForProduct[0]]['ENDMARKETINGDATE']
str(dd)[:]
correction = "2"+str(dd)[1:]
product.loc[arrayIndexInvalidEndMarketingDateForProduct[0],'ENDMARKETINGDATE']= correction

In [35]:
#Nous allons supprimer les lignes incohérentes dans la table package qui ne peuvent pas être corrigées 
arrayIndexToRemoveInPackage = arrayIndexDatesIncoherentesForPackage + arrayIndexIncoherentProductNDCAndNDCPackageCode + arrayIndexIncoherentProductNDCAndProductIdForPackage + arrayIndexInvalidNDCPackageCode + arrayIndexInvalidProductIdForPackage + arrayIndexInvalidStartMktgDateForPackage

arrayIndexToRemoveInPackage = list(set(arrayIndexToRemoveInPackage))

package = package.drop(package.index[arrayIndexToRemoveInPackage])

In [36]:
#Nous allons supprimer les lignes incohérentes dans la table product qui ne peuvent pas être corrigées 
arrayIndexToRemoveInProduct = arrayIndexIncoherentProductNDCAndProductIdForProduct + arrayIndexInvalidEndMarketingDateForProduct + arrayIndexInvalidProductIdForProduct + arrayIndexInvalidProductNDCForProduct

arrayIndexToRemoveInProductUnique = list(set(arrayIndexToRemoveInProduct))

product = product.drop(product.index[arrayIndexToRemoveInProductUnique])

In [38]:
"""#Correction des ProductNDC et NDCPackageCode, nous avons ajuster les erreurs du type 05-juin->0006-0005. Il y en avait 19
for ind in arrayIndexIncoherentProductNDCAndNDCPackageCode:
    if package.iloc[ind]["PRODUCTNDC"] == package.iloc[ind]["NDCPACKAGECODE"][0:9]:
        arrayIndexIncoherentProductNDCAndNDCPackageCode.remove(ind)

#Nous avons le cas interessant ici ou semble y avoir une erreur dasn NDCPACKAGECODE qui entraine une erreur dasn ProductID
#PRODUCTID         0220-2997_8590fcc9-c0c3-f43b-e053-2991aa0a692e
#PRODUCTNDC                                             0220-2997
#NDCPACKAGECODE                                      0220-1997-41
#Name: 6913, dtype: object
#correction de cette erreur dans les 3 attributs
code = package.iloc[6913]["NDCPACKAGECODE"][:9]
package.loc[6913,"PRODUCTNDC"]= code
new = code+package.iloc[6913]["PRODUCTID"][9:]
package.loc[6913,"PRODUCTID"]= new"""

'#Correction des ProductNDC et NDCPackageCode, nous avons ajuster les erreurs du type 05-juin->0006-0005. Il y en avait 19\nfor ind in arrayIndexIncoherentProductNDCAndNDCPackageCode:\n    if package.iloc[ind]["PRODUCTNDC"] == package.iloc[ind]["NDCPACKAGECODE"][0:9]:\n        arrayIndexIncoherentProductNDCAndNDCPackageCode.remove(ind)\n\n#Nous avons le cas interessant ici ou semble y avoir une erreur dasn NDCPACKAGECODE qui entraine une erreur dasn ProductID\n#PRODUCTID         0220-2997_8590fcc9-c0c3-f43b-e053-2991aa0a692e\n#PRODUCTNDC                                             0220-2997\n#NDCPACKAGECODE                                      0220-1997-41\n#Name: 6913, dtype: object\n#correction de cette erreur dans les 3 attributs\ncode = package.iloc[6913]["NDCPACKAGECODE"][:9]\npackage.loc[6913,"PRODUCTNDC"]= code\nnew = code+package.iloc[6913]["PRODUCTID"][9:]\npackage.loc[6913,"PRODUCTID"]= new'

# Question 4

Nous allons chercher à compléter les données manquantes dans les deux tables. Tout d'abord nous allons le faire dans la table package. **Nous n'esseyerons pas de spéculer sur les règles d'inférences entre les attributs, et nous préférons supprimer les lignes contenant des données manquantes.**

**Table package**

- Correction du productNDC à partir du productId car nous avons aucune données manquantes pour l'attribut productId.
- Pour les valeurs manquantes du NDCPackageCode nous n'avons pas accès au *package code segments of the national drug code number*
- Pour les endMarketingDate nous ne pourrons corriger parce que nous ne pouvons pas faire d'inférence quant à la endMarketingDate à partir des données disponibles.

**Table product**

- Correction impossible pour le productId puisque nous n'avons pas accès au *SPL document ID* 
- Correction des valeurs manquantes de proprietaryName si nous connaissons proprietaryNameSuffix
- Quant aux autres attributs nous ne pouvons pas corriger les valeurs manquantes puisque nous ne pouvons faire d'inférence à partir des données disponibles.

# Question 5

# Question 6

Nous allons fusionner nos deux tables en fonction de l'attribut producId. Nous suivons les informations de la documentation : *It is included to help prevent duplicate rows from appearing when joining the product and package files together.*